# EdisonScientific platform client usage example

In [ ]:
import sys
from pathlib import Path

sys.path.insert(0, str(Path("..").resolve()))

In [ ]:
from ldp.agent import AgentConfig

from edison_client import EdisonClient, JobNames
from edison_client.models import (
    AuthType,
    RuntimeConfig,
    Stage,
    TaskRequest,
)

## Client instantiation

Here we use `auth_type=AuthType.API_KEY` to authenticate with the platform.
Please log in to the platform and go to your user settings to get your API key.

In [ ]:
client = EdisonClient(
    stage=Stage.PROD,
    auth_type=AuthType.API_KEY,
    api_key="your-api-key",
)

## Submit a job

Submitting jobs is done by calling the `run_tasks_until_done` method, which receives a `TaskRequest` object.

In [ ]:
task_data = TaskRequest(
    name=JobNames.from_string("crow"),
    query="What is the molecule known to have the greatest solubility in water?",
)
task_response = client.run_tasks_until_done(task_data)

print(f"Job status: {task_response.status}")
print(f"Job answer: \n{task_response.formatted_answer}")

You can also pass a `runtime_config` to the task, which will be used to configure the agent on runtime.
Here, we will define a agent configuration and pass it to the task. This agent is used to decide the next action to take.
We will also use the `max_steps` parameter to limit the number of steps the agent will take.

In [ ]:
agent = AgentConfig(
    agent_type="SimpleAgent",
    agent_kwargs={
        "model": "gpt-4o",
        "temperature": 0.0,
    },
)
task_data = TaskRequest(
    name=JobNames.LITERATURE,
    query="How many moons does earth have?",
    runtime_config=RuntimeConfig(agent=agent, max_steps=10),
)
task_response = client.run_tasks_until_done(task_data)

print(f"Job status: {task_response.status}")
print(f"Job answer: \n{task_response.formatted_answer}")

# Continue a job

The platform allows to keep asking follow-up questions to the previous job.
To accomplish that, we can use the `runtime_config` to pass the `job_id` of the previous job.

Notice that `create_job` accepts both a `JobRequest` object and a dictionary with keywords arguments.

In [ ]:
task_data = TaskRequest(
    name=JobNames.LITERATURE, query="How many species of birds are there?"
)

task_response = client.run_tasks_until_done(task_data)

print(f"First job status: {task_response.status}")
print(f"First job answer: \n{task_response.formatted_answer}")

In [ ]:
continued_job_data = {
    "name": JobNames.LITERATURE,
    "query": "From the previous answer, specifically,how many species of crows are there?",
    "runtime_config": {"continued_job_id": task_response.task_id},
}

continued_task_response = client.run_tasks_until_done(continued_job_data)


print(f"Continued job status: {continued_task_response.status}")
print(f"Continued job answer: \n{continued_task_response.formatted_answer}")